In [6]:
pip install google-cloud-bigquery

  Using cached google_cloud_bigquery-3.11.4-py2.py3-none-any.whl (219 kB)
  Using cached proto_plus-1.22.3-py3-none-any.whl (48 kB)
  Using cached google_cloud_core-2.3.3-py2.py3-none-any.whl (29 kB)
  Using cached google_resumable_media-2.5.0-py2.py3-none-any.whl (77 kB)
  Using cached grpcio_status-1.56.2-py3-none-any.whl (5.1 kB)
  Using cached google_crc32c-1.5.0-cp310-cp310-win_amd64.whl (27 kB)
  Using cached protobuf-4.23.4-cp310-abi3-win_amd64.whl (422 kB)
     ---------------------------------------- 4.2/4.2 MB 4.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.49.1
    Uninstalling grpcio-1.49.1:
      Successfully uninstalled grpcio-1.49.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.0.32 requires protobuf<=3.20.3,>=3.11.0, but you have protobuf 4.23.4 which is incompatible.
tensorboard 2.11.0 requires protobuf<4,>=3.9.2, but you have protobuf 4.23.4 which is incompatible.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.4 which is incompatible.
tensorflow-metadata 1.11.0 requires protobuf<4,>=3.13, but you have protobuf 4.23.4 which is incompatible.

[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install db-dtypes

  Using cached db_dtypes-1.1.1-py2.py3-none-any.whl (14 kB)
  Using cached pyarrow-12.0.1-cp310-cp310-win_amd64.whl (21.5 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
from google.cloud import bigquery
import pandas

# Set up BigQuery client with explicit project ID
bq_client = bigquery.Client(project='stackai-394819')

# Set up query
query = """
    SELECT *
    FROM `stackai-394819.StackAI.posts_cleaned`
"""

# Run query and fetch results into pandas DataFrame
df = bq_client.query(query).to_dataframe()


In [44]:
# Concatenate relevant columns for embedding, handling NaN values
relevant_text = df.apply(lambda row: ' '.join(filter(lambda x: pd.notna(x), [row['question_title'], row['question_body'] , row['accepted_answer']])), axis=1)

# Convert relevant_text to a list
relevant_text_list = relevant_text.tolist()

# Load the pre-trained Sentence Transformers model
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

def generate_embeddings(text):
    embeddings = model.encode(text)
    return embeddings

# Generate embeddings for relevant data
data_embeddings = generate_embeddings(relevant_text_list)

# Add new column to DataFrame with embeddings
df['embeddings'] = data_embeddings.tolist()

In [45]:
df['embeddings']

0      [-0.011609231121838093, 0.37894147634506226, 0...
1      [-0.6155633330345154, 0.24588115513324738, 0.0...
2      [-0.25975582003593445, 0.4285120368003845, 0.2...
3      [-0.8276649713516235, 0.3956160545349121, 0.14...
4      [-0.32777541875839233, -0.05247429013252258, 0...
                             ...                        
495    [-1.0283185243606567, 0.6513798236846924, 0.00...
496    [-0.33344218134880066, 0.2886114716529846, 0.2...
497    [-0.41436970233917236, 0.6520484685897827, 0.5...
498    [-0.2258448600769043, 0.6035326719284058, 0.28...
499    [-0.6599689722061157, 0.8397446870803833, 0.50...
Name: embeddings, Length: 500, dtype: object

In [46]:
# Update BigQuery table with new column
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
    schema=[
        bigquery.SchemaField("embeddings", "STRING"),
    ],
)

job = bq_client.load_table_from_dataframe(
    df, 'stackai-394819.StackAI.posts_cleaned', job_config=job_config
)
job.result()

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:603: UserWarning: Pyarrow could not determine the type of columns: accepted_answer_view_count.
  warnings.warn(


LoadJob<project=stackai-394819, location=US, id=7f0a479b-c31a-45b4-a7b5-ed192e0bf8bc>

In [20]:
df.head()

,question_id,question_title,question_body,question_tags,accepted_answer,question_creation_date,accepted_answer_creation_date,accepted_answer_owner_display_name,accepted_answer_score,accepted_answer_view_count
0,2989926,Can I chain database links in Oracle?,"<p>I have 3 databases. 1 links to 2, 2 links ...",sql|oracle|database-link,<p>I think you can do this by creating synonym...,2010-06-07 13:31:03.560000+00:00,2010-06-07 13:38:27.300000+00:00,Thomas Jones-Low,12,None
1,479408,Execute Statement or Run Script?,<p>While entering a SQL statement in Oracle SQ...,sql|database|oracle|oracle-sqldeveloper,<p><kbd>Run Statement</kbd> will give you a li...,2009-01-26 10:58:09.917000+00:00,2009-01-26 11:10:41.703000+00:00,Greg Reynolds,30,None
2,843242,Dropping then adding a constraint fails in oracle,<p>I'm trying to move a primary key constraint...,sql|oracle,<p>If the original constraint was a primary ke...,2009-05-09 12:46:39.020000+00:00,2009-05-09 13:36:33.950000+00:00,Thomas Jones-Low,11,None
3,1692538,"What is the difference between primary, unique...",<p>What is the difference between <code>primar...,sql-server|database|oracle|constraints|indexing,<p>Primary Key and Unique Key are Entity integ...,2009-11-07 09:25:06.003000+00:00,2009-11-07 09:37:16.537000+00:00,Nadir SOUALEM,26,None
4,626267,What Situations Cause Oracle Packages to Becom...,<p>The scenario that created this question: <...,oracle|plsql|package|dependencies,<p>Changing anything object that a package rel...,2009-03-09 13:37:48.747000+00:00,2009-03-09 14:46:01.923000+00:00,Thomas Jones-Low,15,None


In [21]:
df.dtypes

question_id                                         Int64
question_title                                     object
question_body                                      object
question_tags                                      object
accepted_answer                                    object
question_creation_date                datetime64[ns, UTC]
accepted_answer_creation_date         datetime64[ns, UTC]
accepted_answer_owner_display_name                 object
accepted_answer_score                               Int64
accepted_answer_view_count                         object
dtype: object

In [22]:
df.shape

(500, 10)

In [23]:
from google.cloud import bigquery
import pandas

# Set up BigQuery client with explicit project ID
bq_client = bigquery.Client(project='stackai-394819')

# Set up query
query = """
    SELECT *
    FROM `stackai-394819.StackAI.comments_cleaned`
"""

# Run query and fetch results into pandas DataFrame
df2 = bq_client.query(query).to_dataframe()

In [24]:
df2.head()

,post_id,text,creation_date,score
0,31124058,thanks but that doesn't show how to enable out...,2015-06-29 20:16:12.830000+00:00,0
1,31124058,You are correct - it is the 'bulb' icon! I ca...,2015-06-29 20:17:22.557000+00:00,0
2,30437541,"@Ben This issue was with Data formats, I chang...",2015-07-03 09:12:24.610000+00:00,0
3,31532522,You don't have any values defined for `n` in t...,2015-07-21 07:15:44.393000+00:00,0
4,31532522,It would be nice if you showed what the result...,2015-07-22 04:58:03.200000+00:00,0


In [26]:
df2.dtypes

post_id                        Int64
text                          object
creation_date    datetime64[ns, UTC]
score                          Int64
dtype: object

In [27]:
df2.shape

(1215, 4)

In [12]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.llms.openai import OpenAI
from langchain.chains.summarize import load_summarize_chain
from scipy.spatial.distance import cdist
import openai

In [13]:
# Load the pre-trained Sentence Transformers model
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [15]:
# Preprocessing function to convert tags from string to list
def process_tags(Tags):
    if pd.isna(Tags):  # Handle NaN values
        return []      # Return an empty list for NaN values
    return Tags.split(',')

# Apply the preprocessing to the 'Tags' column (uppercase "T")
df['question_tags'] = df['question_tags'].apply(process_tags)

def generate_embeddings(text):
    embeddings = model.encode(text)
    return embeddings

def filter_data(user_input_tag):
    filtered_data = df

    # Keep only data for the user input tag
    filtered_data = filtered_data[filtered_data['question_tags'].apply(lambda tags: user_input_tag in tags)]

    return filtered_data

In [47]:
def topic_relevance_search(user_input, user_input_tag):
    # Filter data based on user input tag and "PostTypeId"
    filtered_data = filter_data(user_input_tag)

    # Check if there is relevant data for the user input tag
    if filtered_data.empty:
        print(f"No relevant data found for the tag '{user_input_tag}'.")
        return []

    # Fetch embeddings from BigQuery table
    data_embeddings = np.array(filtered_data['embeddings'].tolist())

    # Generate embedding for user input
    user_embedding = generate_embeddings([user_input])

    # Calculate cosine similarity between user input and data embeddings
    similarity_scores = cosine_similarity(user_embedding, data_embeddings)[0]

    # Sort the data by similarity scores in descending order and select top 5 rows
    top_indices = np.argsort(similarity_scores)[::-1][:5]
    
    top_similar_topics = filtered_data.iloc[top_indices][['question_id', 'question_title', 'question_body', 'accepted_answer']].to_dict('records')
    
    top_similarity_scores = similarity_scores[top_indices]

    return top_similar_topics, top_similarity_scores


In [49]:
if __name__ == "__main__":
    # Example usage:
    user_question = "How to setup oracle database?"
    user_tag = "sql|oracle"
    

    similar_topics, similarity_scores = topic_relevance_search(user_question, user_tag)
    
    print("Most similar topics:")
    
    for i in range(len(similar_topics)):
        question_id, question_title, question_body, accepted_answer = similar_topics[i]['question_id'], similar_topics[i]['question_title'], similar_topics[i]['question_body'], similar_topics[i]['accepted_answer']
        
        print(f"{i+1}. Question ID: {question_id} : {question_title} (similarity score: {similarity_scores[i]:.2f})")

Most similar topics:
1. Question ID: 1371192 : Oracle: find index creation date from systables/information_schema? (similarity score: 0.70)
2. Question ID: 19926959 : How to get root by recursive oracle query in Hibernate (similarity score: 0.67)
3. Question ID: 38267997 : create sql Database from ER diagram (similarity score: 0.66)
4. Question ID: 21990025 : What is the Default Date Format in Oracle? (similarity score: 0.65)
5. Question ID: 14590344 : Oracle SQLDeveloper: How to search which tables have a column matching given query? (similarity score: 0.64)


In [32]:
selected_topic_question_id = 843242
    
selected_topic_index = next(i for i in range(len(similar_topics)) if similar_topics[i]['question_id'] == selected_topic_question_id)
    
selected_topic = similar_topics[selected_topic_index]
    
question_id, question_title, question_body, accepted_answer = selected_topic['question_id'], selected_topic['question_title'], selected_topic['question_body'], selected_topic['accepted_answer']
    
print(f"\nSelected topic: {question_title}")
    
if pd.isna(accepted_answer):
        print("No accepted answer")
        accepted_answer_text = ""
else:
        print(f"Accepted answer: {accepted_answer}")
        accepted_answer_text = f"\n\nAccepted answer:\n{accepted_answer}"
    
    
post_comments = df2[df2['post_id'] == question_id]
    
comments_text = ""
if not post_comments.empty:
        print("Comments:")
        for comment in post_comments['text']:
            print(f"\t- {comment}")
            comments_text += f"\n- {comment}"
    


Selected topic: Dropping then adding a constraint fails in oracle
Accepted answer: <p>If the original constraint was a primary key constraint, Oracle creates an index to enforce the constraint. This index has the same name as the constraint (C_NAME in your example). You need to drop the index separately from the constraint. So you will need to do a :</p>

<pre><code>ALTER TABLE &lt;table1&gt; DROP CONSTRAINT C_NAME;
DROP INDEX C_NAME;

ALTER TABLE &lt;table1&gt; ADD CONSTRAINT C_NAME PRIMARY KEY
( COLUMN_2 ) ENABLE;
</code></pre>
Comments:
	- You need to specify the table name somewhere along the line :D


In [34]:
class Document:
    def __init__(self, page_content):
        self.page_content = page_content
        self.metadata = {}

In [35]:
data = f"{question_title}{question_body}{accepted_answer}\n\nComments:{comments_text}"
# Create a Document object from the data string
doc = Document(page_content=data)

In [38]:
# Initialize the OpenAI module and load the summarize chain
llm = OpenAI(temperature=0, openai_api_key="")
chain = load_summarize_chain(llm, chain_type="stuff")

# Use the string variable as input to the summarization chain
summary = chain.run(input_documents=[doc], question="Write a concise summary within 300 words.")
#print summary of post title
print(f"summary of the post - {question_title} : ", summary)

summary of the post - Dropping then adding a constraint fails in oracle :   When trying to move a primary key constraint to a different column in Oracle, the attempt fails with an error saying the constraint already exists. To fix this, the index associated with the constraint must be dropped separately from the constraint before adding the new constraint. The table name must also be specified.


In [40]:
openai_api_key = ""
# User input
user_input = "Dropping then adding a constraint fails in oracle"
# Concatenate the data: post_title, post_text, accepted_answer_text, and comments_text
data = f"{question_title}{question_body}{accepted_answer}\n\nComments:{comments_text}"
openai.api_key = openai_api_key

In [41]:
def get_openai_response(user_input, data):
    # Define the conversation history with system, user, and data messages
    conversation = f"You are a helpful assistant.\n\nUser: {user_input}\n\nData: {data}"

    # Use the OpenAI API to generate a response
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Give the answer to the user input using the data. If the answer doesn't exist in data, create your own answer."},
            {"role": "user", "content": user_input},
            {"role": "system", "content": f"Data: {data}"}
        ],
        temperature=0.2,
        max_tokens=500
    )

    # Extract and return the model-generated answer from the response
    return response["choices"][0]["message"]["content"].strip()

In [42]:
# Get the model's response based on the user input and the given data
answer = get_openai_response(user_input, data)

# Print the generated answer
print("AI: " + answer)

AI: If the original constraint was a primary key constraint, Oracle creates an index to enforce the constraint. This index has the same name as the constraint. You need to drop the index separately from the constraint. So you will need to do a:

```
ALTER TABLE <table_name> DROP CONSTRAINT <constraint_name>;
DROP INDEX <index_name>;

ALTER TABLE <table_name> ADD CONSTRAINT <constraint_name> PRIMARY KEY (<column_name>) ENABLE;
```

Make sure to replace `<table_name>`, `<constraint_name>`, `<index_name>`, and `<column_name>` with the appropriate values for your situation.
